In [ ]:
# 查看环境安装情况
!pip freeze | grep tensor

In [ ]:
# 查看文件行数
!wc -l data/poetry/*.txt

In [ ]:
# 查看文件内容
!head -5 data/poetry/*.txt

In [ ]:
# 多文件操作
with open('data/poetry/raw.txt', 'r') as rawfp,\
  open('data/poetry/input.txt', 'w') as infp,\
  open('data/poetry/output.txt', 'w') as outfp:
    
    prev_line = ''
    for curr_line in rawfp:
        curr_line = curr_line.strip()
        # poems break at empty lines, so this ensures we train only
        # on lines of the same poem
        if len(prev_line) > 0 and len(curr_line) > 0:       
            infp.write(prev_line + '\n')
            outfp.write(curr_line + '\n')
        prev_line = curr_line

In [ ]:
#github忽略文件
touch .gitignore
vi .gitignore
# 忽略文件夹
/doc/

In [ ]:
# 合并checkpoints
python avg_checkpoints.py --checkpoints=[tacotron_model.ckpt-345000,tacotron_model.ckpt-350000] --output_path="/Users/liuxinxin/Downloads/new.ckpt"

In [ ]:
# tf.reduce_mean 说明
a = tf.random_uniform([2,3],minval=5,
    maxval=5,)
# [[5. 5. 5.]
#  [5. 5. 5.]]
b = tf.reduce_mean(a,-1)
# [5. 5.]
b = tf.reduce_mean(a)
# 5.0
b = tf.reduce_mean(a,-1,keepdims=True)
# [[5.]
#  [5.]]

In [ ]:
# 使用request访问api
import requests

sentences = "测试语句"
url = 'http://127.0.0.1:19877/?sentences=' + sentences
crawl_content = requests.get(url)
crawl_content.text

In [ ]:
# 异步访问
async def call_coroutine(session, url,i,files):
    with async_timeout.timeout(60):
        async with session.get(url) as response:
            content = await response.text()
            content = json.loads(content)
            files[i] = content['wav_file']
            print(content['wav_file'])
            print(i)
            return await response.release()
 
 
async def main(loop,sentences,files):
    async with aiohttp.ClientSession(loop=loop) as session:
        tasks = [call_coroutine(session, 'http://127.0.0.1:8099/?sentences=' + sentence,i,files) for i,sentence in enumerate(sentences)]
        await asyncio.gather(*tasks)


def synic(sentences):
    start = time()
    files = {}
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main(loop,sentences,files))
    files  = sorted(files.items(),key = lambda item:item[0])
    files = [file[1]+'.wav'  for file in files]
    if len(files) > 1:
        break_wav = ["480ms.wav"] * (len(files) - 1)
        j = 1
        for i, ele in enumerate(break_wav):
            files.insert(i + j, ele)
            j += 1
        files = ['wav_out/' + filename for filename in files]
        cbn = sox.Combiner()
        nowTime = datetime.now().strftime('%Y%m%d%H%M%S')
        output_file = 'wav-' + nowTime + '.wav'
        cbn.build(files, 'wav_out/' + output_file, 'concatenate')
    else:
        output_file = files[0]
    stop = time()
    print(stop - start)
    return output_file

In [ ]:
# 设置互信
# 生成秘钥
ssh-keygen -t rsa
# 把id_rsa.pub内容写入authorized_keys中
# 修改权限
chmod 700 .ssh #.ssh权限
chmod 600 authorized_keys

In [ ]:
# ubuntu 压缩文件
 tar -zcvf workspace.tar.gz worksapce

In [1]:
# 生成传输数据

dict_server_dir = {
    "align":"/home/zpf/exp/ta-2/logs-Tacotron/eval-dir/plots",
    "model" : "/home/zpf/exp/ta-2/logs-Tacotron/taco_pretrained",
    "usr" : "/usr",
    "wav" : "/home/zpf/exp/ta-2/logs-Tacotron/eval-dir/wavs"
}

dict_file = {
    "align" : "step-60000-eval-align.png",
    "wav" : "step-35000-eval-wave-from-linear.wav",
    "model" : "checkpoint.zip"
}

dict_server = {
    "zpf" : {
        "admin" : "zpf",
        "address" : "124.126.217.105"
    },
    "206" : {
        "admin" : "root",
        "address" : "112.126.80.206"
    }
}

server = dict_server["zpf"]

server_admin = server["admin"]
server_address = server["address"]
server_dir = dict_server_dir["model"]
home_dir = "/Users/liuxinxin/Downloads"
file_name = dict_file["model"]


send_command = "scp -P 22 "+ home_dir + "/" + file_name + " " + server_admin + "@" + server_address + ":" + server_dir + "/"
rece_command = "scp -P 22 "+ server_admin + "@" + server_address + ":" + server_dir+ "/" + file_name + " " + home_dir + "/"

print("send command:")
print(send_command)
print("rece_command:")
print(rece_command)

send command:
scp -P 22 /Users/liuxinxin/Downloads/checkpoint.zip zpf@124.126.217.105:/home/zpf/exp/ta-2/logs-Tacotron/taco_pretrained/
rece_command:
scp -P 22 zpf@124.126.217.105:/home/zpf/exp/ta-2/logs-Tacotron/taco_pretrained/checkpoint.zip /Users/liuxinxin/Downloads/


In [2]:
!$rece_command

checkpoint.zip                                100%  303MB   2.2MB/s   02:17    


In [7]:
!$send_command

4.zip                                         100%  622MB   2.0MB/s   05:17    


In [ ]:
# 查看端口占用情况
lsof -i tcp:8099

In [ ]:
# json不能传输ndarray的问题，如果解决
import numpy as np
import json

# ndarray 无法序列化
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

a = np.array([1, 2, 3])
print(json.dumps({'aa': [2, (2, 3, 4), a], 'bb': [2]}, cls=NumpyEncoder))

In [ ]:
# mac sh 文件可以执行
chmod u+x

In [ ]:
# 大文件查询
du --max-depth=1 -h .

In [ ]:
# 安装redis
下载https://redis.io/download
$ cd redis-3.2.8
$ make
$ sudo make install 
# 运行
/usr/local/bin
redis-server

In [8]:
# 使用redis 和 删除key
import redis   # 导入redis模块，通过python操作redis 也可以直接在redis主机的服务端操作缓存数据库

r = redis.Redis(host='localhost', port=6379, decode_responses=True)   # host是redis主机，需要redis服务端和客户端都启动 redis默认端口是6379
# r.set('name', 'junxi')  # key是"foo" value是"bar" 将键值对存入redis缓存
# print(r['name'])
# print(r.get('name'))  # 取出键name对应的值
r.hset(name, key, value) 
print(type(r.get('name')))

keys = r.keys()
r.delete(*keys)
r.keys()

junxi
junxi
<class 'str'>


[]

In [7]:
sentences = "五千&四百&三十三万&五千&八百&二十一"

hasMS=False
sentences=sentences.split("&")
if(len(sentences)>0):
    hasMS=False
else:
    sentences=sentences.split("。")
    hasMS=True

    
print(hasMS)
print(sentences)

False
['五千', '四百', '三十三万', '五千', '八百', '二十一']


In [2]:
# 分割句号和逗号
strs = '你好，我是深度学习工程师。他是java工程师，而且还是单身。'
arrays = list(filter(None,strs.split('。')))
fuhao = []
chars = []
for line in arrays:
    lines = list(filter(None,line.split('，')))
    if len(lines) > 1:
        for i in range(len(lines)-1):
            chars.append('，')
        chars.append('。')
    else:
        chars.append('。')
    fuhao.extend(lines)
print("符号：",chars)
print("文字：",fuhao)

符号： ['，', '。', '，', '。']
文字： ['你好', '我是深度学习工程师', '他是java工程师', '而且还是单身']


In [5]:
# 分割句号和逗号
import re

sentences = "你好，我是深度学习工程师。他是java工程师，而且还是单身。"
fields = re.split(r'(。|，|)', sentences)
values =  list(filter(None,fields[::2]))
delimiters = fields[1::2]

print("符号：",delimiters)
print("文字：",values)

符号： ['，', '。', '，', '。']
文字： ['你好', '我是深度学习工程师', '他是java工程师', '而且还是单身']


/Users/liuxinxin/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [11]:
# 合并结果
word = []
for value,delimiter in zip(values,delimiters):
    word.append(value)
    break_value = "0"
    if delimiter == '，':
        break_value = '1'
    word.append(break_value)
word = ''.join(word)
print(word)

你好1我是深度学习工程师0他是java工程师1而且还是单身0


In [17]:
# MD5生成
data = "问题解决了，但是心头的疑问却更多了。为什么不能直接hashlib.md5(data)，非要强制进行编码转换，设计者的初衷何在？中文字符在Python中是以什么形式存在？说明中文字符在Python中是以unicode存在的。至此，所有的疑问都得以解除了。在hash前要求进行编码转换，是因为同一个字符串在不同的编码体系下有不同的值，为确保不发生歧义必须要进行一次显性转换。"
m = hashlib.md5(data.encode())
print(m.hexdigest())

67f12172333b84b312b36a247d8d0ce1


In [44]:
# 手工实现矩阵算法
import numpy as np
def elementwise_multiplication(vec_a, vec_b):
    vec_result = np.zeros(vec_a.shape)
    for i,item in enumerate(zip(vec_a,vec_b)):
        c = item[0] * item[1]
        vec_result[i] = c
    return vec_result

def elementwise_addition(vec_a, vec_b):
    vec_result = np.zeros(vec_a.shape)
    for i,item in enumerate(zip(vec_a,vec_b)):
        c = item[0] + item[1]
        vec_result[i] = c
    return vec_result

def vector_sum(vec_a):
    result = 0
    for a in vec_a:
        result += a
    return result

def vector_average(vec_a):
    a_sum = vector_sum(a)
    average = a_sum / len(a)
    return average

def dot_product(vec_a, vec_b):
    elementwise = elementwise_multiplication(vec_a, vec_b)
    result = vector_sum(elementwise)
    return result

In [ ]:
# 查看某个进程的资源占用
top -c -p $(pgrep -d',' -f guni)